<a href="https://colab.research.google.com/github/Ankit0207/Big-Data-Analytics/blob/master/IDS561lab3_2020spr(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#                     Spark Data Frame Transformations and Actions

In [0]:
!wget -q http://apache.mirrors.hoobly.com/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
#Untar the Spark installer
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz

In [3]:
!ls 

sample_data  spark-2.4.5-bin-hadoop2.7	spark-2.4.5-bin-hadoop2.7.tgz


# Install findspark - a python library to find Spark

In [0]:
!pip install -q findspark

# Set environment variables
Set Java and Spark home based on the location where they are stored

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Creat a local Spark session

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Begin of Spark Data frame Transformations Program

In [7]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
#Changing the directory to the required path
%cd /content/drive/My Drive/Big Data
!ls 

/content/drive/My Drive/Big Data
Amazon_Responded_Oct05.csv  spark-2.4.5-bin-hadoop2.7
experiment.txt		    spark-2.4.5-bin-hadoop2.7.tgz


#TASK 1 :

#Create a dataframe “daily_active_users”. Find out the users who are active in at least five listed days 

#i.e. created posts in at least 5 days) in Amazon_Responded_Oct05.csv and save their “user_screen_name” and “user_id_str” in the dataframe.

In [0]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/Big Data/Amazon_Responded_Oct05.csv", sep=",", error_bad_lines=False)

In [0]:
#Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

#Then convert Pandas Dataframe to Spark Dataframe
dfs = spark.createDataFrame(df)

In [98]:
import pyspark
from pyspark.sql import functions
import pyspark.sql.functions as F

#Modify the tweet_created_at column by taking only the date i.e. Month Day and Year
#i.e. Tue Nov 01 01:57:25 +0000 2016 is converted to Nov 01 2016

new_df = dfs.withColumn('tweet_created_at', F.concat(dfs.tweet_created_at.substr(5, 6), dfs.tweet_created_at.substr(26, 5)))
new_df.select("tweet_created_at").show()

+----------------+
|tweet_created_at|
+----------------+
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
|     Nov 01 2016|
+----------------+
only showing top 20 rows



In [0]:
import pyspark.sql.functions as F
#Below operation is done to Find out the users who created posts in at least 5 days.
new_df1 = new_df.groupBy("user_screen_name").agg(F.countDistinct("tweet_created_at"))

In [73]:
#Filters the dataframe based on the users who have created atleast 5 twitter posts
new_df2 = new_df1.filter("count(DISTINCT tweet_created_at) >= 5")

new_df2.show()

+----------------+--------------------------------+
|user_screen_name|count(DISTINCT tweet_created_at)|
+----------------+--------------------------------+
|   ItsJustMe_Rae|                               6|
|  Green_JamesBee|                               6|
|   That_Musician|                               5|
| PritiYa80985097|                              10|
|    Speechlesstx|                               5|
|    Kevin_Ellen_|                               5|
|      nsvaluto74|                               5|
|        RGengage|                               9|
| sureshmuthrotil|                               6|
|   ajaygupta1974|                               5|
|       Tushi_Joy|                               7|
|      lupitasahu|                               6|
| SardarGurvinde2|                               5|
|        RISSHHII|                               5|
| Rajeshk76856612|                               5|
|  dahnamchandler|                               8|
|    TaiKami

In [0]:
#Filter the dataframe by keeping only the users obtained in the new_df2 dataframe
new_df3 = new_df.join(new_df2, ["user_screen_name"], "right")

In [0]:
from pyspark.sql.types import DecimalType

#Create a new Spark dataframe keeping only the below columns
daily_active_users = new_df3.select("user_screen_name", "user_id_str")

#Below code converts the exponential column type to Numeric format i.e. 4.7E38491 to 466384851
#DecimalType(18,0) represents 18 digits to the left of the decimal and 0 digits to the right of the decimal

daily_active_users = daily_active_users.withColumn("user_id_str", daily_active_users.user_id_str.cast(DecimalType(18, 0)))

In [76]:
#daily_active_users contains the user name and user id of all the users who created Twitter posts in atleast 5 days
daily_active_users.select('*').show()

+----------------+-----------+
|user_screen_name|user_id_str|
+----------------+-----------+
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
|  Green_JamesBee|  466384851|
+----------------+-----------+
only showing top 20 rows



#TASK 2 :

#A company would like to conduct an A/B test on Twitter. The experiment.txt file includes the user_id_str they selected as potential experiment targets. 

#Please create a dataframe “experiment_user” to document the select user id and whether they are active users (join the dataframe from step 1). 

#Then calculate the percentage of active user and print out the result.

In [0]:
exp_txt = pd.read_csv("/content/drive/My Drive/Big Data/experiment.txt", sep=",", header=None,error_bad_lines=False)

#Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

#Then convert Pandas Dataframe to Spark Dataframe
exp_df = spark.createDataFrame(exp_txt)

#Rename the column to id_str
exp_df = exp_df.withColumnRenamed("0", "user_id_str")

In [0]:
exp_df1 = daily_active_users.join(exp_df, ["user_id_str"], how="full_outer")

In [0]:
#cleaned_data contains the unique user_id_str records
cleaned_data = exp_df1.drop_duplicates(['user_id_str'])

#cleaned_data contains the unique user_id_str records
duplicate_data = exp_df1.exceptAll(cleaned_data)

In [80]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import *
#The below code will count the number of times user_id_str is present and will give it a count in a separate count column
count_data = exp_df1.groupBy(["user_id_str"]).agg(F.count(F.lit(1)).alias('count'))

#This part of code will replace count value with "yes" if count > 1 else "no"
experiment_user = count_data \
                             .withColumn("count",\
                              when(count_data["count"] > 1, "yes")\
                             .otherwise("no"))

#Changing the count column name to "Whether_active"                            
experiment_user = experiment_user.withColumnRenamed("count", "Whether_active")
experiment_user.show()                             

+-----------+--------------+
|user_id_str|Whether_active|
+-----------+--------------+
|   23068174|           yes|
|   24038624|            no|
|   25805398|            no|
|   38289347|           yes|
|   39395899|           yes|
|   54311555|            no|
|   71308901|            no|
|   72342053|            no|
|   84303818|            no|
|  124404304|            no|
|  153994363|            no|
|  241391028|           yes|
|  263063213|            no|
|  293677963|           yes|
|  351123880|            no|
|  517127347|            no|
|  595548720|           yes|
|  712609620|            no|
|  842317388|            no|
|  911933083|            no|
+-----------+--------------+
only showing top 20 rows



In [94]:
#Calculate the percentage of active user and print out the result.

active = experiment_user.filter("Whether_active == 'yes'")
no_active = active.count() 
total     = experiment_user.count()
active    = (no_active/total) * 100
print(active)

10.825118656443959


In [0]:
#Keep only the user_id_str column and remove other unneccessary columns from the below Spark dataframe

daily_active_users = daily_active_users.drop("user_screen_name")
daily_active_users = daily_active_users.withColumn("active", lit("active"))
daily_active_users = daily_active_users.drop_duplicates(["user_id_str"])

experiment_user    = experiment_user.drop("Whether_active")
experiment_user    = experiment_user.withColumn("experiment", lit("experiment")) 
experiment_user    = experiment_user.drop_duplicates(["user_id_str"])

#TASK 3  :

#To help the company prepare the data, please select the records (all columns) in Amazon_Responded_Oct05.csv when a user_id_str is included in all the 2 dataframes. 

#For example, if the user_id_str from Amazon_Responded_Oct05.csv cannot be found in daily_active_user and experiment_user, you should skip. Save the result in a dataframe and then export it as Amazon_new.csv

In [0]:
#Main Table : dfs
#Table 1    : daily_active_users
#Table 2    : experiment_user

amazon = dfs.join(daily_active_users, ["user_id_str"], "left").join(experiment_user, ["user_id_str"], "left")
amazon = amazon.filter(amazon.active.isNotNull() | amazon.experiment.isNotNull())
amazon = amazon.drop("active", "experiment")

amazon_new = amazon

In [0]:
#Finally Export the result to a .csv file
amazon_new.write.format("csv").save("/content/drive/My Drive/Big Data/Amazon_new.csv")

In [86]:
#=============================================
#Example 3-Table join operations for practice |
#=============================================

one = spark.createDataFrame(
    [
        (1, 'foo'), # create your data here, be consistent in the types.
        (1, 'bar'),
        (3, 'choc'),
        (4, 'lol'),
    ],
    ['id', 'txt'] # add your columns label here
)

two = spark.createDataFrame(
    [
        (13, 'foo1'), # create your data here, be consistent in the types.
        (5, 'bar1'),
        (1, 'choc1'),
        (7, 'lol1'),
    ],
    ['id', 'txt1'] # add your columns label here
)

three = spark.createDataFrame(
    [
        (11, 'foo2'), # create your data here, be consistent in the types.
        (8, 'bar2'),
        (19, 'choc2'),
        (3, 'lol2'),
    ],
    ['id', 'txt2'] # add your columns label here
)

#two = two.drop("txt1")
#three = three.drop("txt2")

combined = one.join(two, ["id"], "left").join(three, ["id"], "left")
combined = combined.filter(combined.txt1.isNotNull() | combined.txt2.isNotNull())
combined = combined.drop("txt1", "txt2")
combined.show()

+---+----+
| id| txt|
+---+----+
|  1| foo|
|  1| bar|
|  3|choc|
+---+----+

